<a target="_blank" href="https://colab.research.google.com/github/OpenProteinAI/openprotein-docs/blob/develop/source/python-api/structure-prediction/Using_Boltz.ipynb">
  <img src="../../_static/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="./Using_Boltz.ipynb">
  <img src="../../_static/download-notebook-badge.svg" alt="Download Notebook"/>
</a>
<a target="_blank" href="https://github.com/OpenProteinAI/openprotein-docs/blob/develop/source/python-api/structure-prediction/Using_Boltz.ipynb">
  <img src="../../_static/view-in-github-badge.svg" alt="View in GitHub"/>
</a>

# Using Boltz
This tutorial demonstrates how to use the Boltz-2 model to predict the
structure of a molecular complex, including proteins and ligands. We
will also show how to request and retrieve predicted binding affinities
and other quality metrics.

# What you need before getting started

First, ensure you have an active `APISession`. Then, import the
necessary classes for defining the components of your complex.

In [1]:
%env OPENPROTEIN_USERNAME=admin
%env OPENPROTEIN_PASSWORD=:open>PROTEIN2

env: OPENPROTEIN_USERNAME=admin
env: OPENPROTEIN_PASSWORD=:open>PROTEIN2


In [2]:
import openprotein
from openprotein.protein import Protein
from openprotein.chains import Ligand

# Login to your session
session = openprotein.connect()

# Defining the Molecules

Boltz-2 can model various molecule types, including proteins, ligands,
DNA, and RNA. For this example, we'll predict the structure of a protein
dimer in complex with a ligand.

We will define a dimer and one ligand. When using Boltz models, we can
specify that a `Protein` is meant to be an oligomer by specifying
multiple ids in the `chain_id`. In this case, the protein is a dimer
since we have `["A", "B"]`.

Note that for affinity prediction, the ligand that is binding must have
a single, unique string for its `chain_id`.

In [2]:
# Define the proteins
proteins = [
    Protein(sequence="MVTPEGNVSLVDESLLVGVTDEDRAVRSAHQFYERLIGLWAPAVMEAAHELGVFAALAEAPADSGELARRLDCDARAMRVLLDALYAYDVIDRIHDTNGFRYLLSAEARECLLPGTLFSLVGKFMHDINVAWPAWRNLAEVVRHGARDTSGAESPNGIAQEDYESLVGGINFWAPPIVTTLSRKLRASGRSGDATASVLDVGCGTGLYSQLLLREFPRWTATGLDVERIATLANAQALRLGVEERFATRAGDFWRGGWGTGYDLVLFANIFHLQTPASAVRLMRHAAACLAPDGLVAVVDQIVDADREPKTPQDRFALLFAASMTNTGGGDAYTFQEYEEWFTAAGLQRIETLDTPMHRILLARRATEPSAVPEGQASENLYFQ"),
]
proteins[0].chain_id = ["A", "B"]

# You can also specify the proteins to be cyclic by setting the property
# proteins[0].cyclic = True

# Define the ligand
# We use the three-letter code for S-adenosyl-L-homocysteine (SAH)
# The chain_id 'C' is the "binder" we will reference later.
ligands = [
    Ligand(ccd="SAH", chain_id="C")
]

# Create MSA for the Protein using Homology Search

When using Boltz with protein sequences, we need to supply an MSA to
help inform the model. Otherwise, we can also explicitly set it to run
using single sequence mode. You have to specify `protein.msa` either an
MSA or to use `Protein.single_sequence_mode`.

Here, we will be building an MSA using our platform capabilities. Take
note of the syntax here: creating an MSA with a complex uses ColabFold's
syntax of joining sequences with `:`.

In [3]:
msa_query = []
for p in proteins:
    if p.chain_id is not None and isinstance(p.chain_id, list):
        for _ in p.chain_id:
            msa_query.append(p.sequence.decode())
    else:
        msa_query.append(p.sequence.decode())
msa = session.align.create_msa(seed=":".join(msa_query))

for p in proteins:
    p.msa = msa
    # If desired, use single sequence mode to specify no msa
    # p.msa = Protein.single_sequence_mode

# Predicting the Complex Structure and Affinity

Now, we can call the `fold` method on the Boltz-2 model.

The key steps are:

1.  Access the model via `session.fold.boltz2`.
2.  Pass the defined proteins and ligands.
3.  To request binding affinity prediction, include the `properties`
    argument. This argument takes a list of dictionaries. For affinity,
    you specify the `binder`, which must match the `chain_id` of a
    ligand you defined.

In [4]:
# Request the fold, including an affinity prediction for our ligand.
fold_job = session.fold.boltz2.fold(
    proteins=proteins,
    ligands=ligands,
    properties=[{"affinity": {"binder": "C"}}]
)
fold_job

FoldJob(num_records=1, job_id='c198b1c9-3faf-4767-b8c2-2f96282d7951', job_type=<JobType.embeddings_fold: '/embeddings/fold'>, status=<JobStatus.PENDING: 'PENDING'>, created_date=datetime.datetime(2025, 8, 12, 23, 44, 36, 940506, tzinfo=TzInfo(UTC)), start_date=None, end_date=None, prerequisite_job_id=None, progress_message=None, progress_counter=0, sequence_length=None)

The call returns a `FoldComplexResultFuture` object immediately. This is
a reference to your job running on the OpenProtein platform. You can
monitor its status or wait for it to complete.

In [5]:
# Wait for the job to finish
fold_job.wait_until_done(verbose=True)

Waiting: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:37<00:00,  3.98s/it, status=SUCCESS]


True

# Retrieving the Results

Once the job is complete, you can retrieve the various outputs from the
future object.

**Getting the Structure File** The primary result is the predicted
structure, which you can retrieve as a mmCIF file. Note that we only implemented mmCIF output format for Boltz.

In [6]:
# Get the result as a PDB bytestring
structure_data = fold_job.get()

# You can now save this to a file
with open("complex_structure.cif", "wb") as f:
    f.write(structure_data)

print('\n'.join(structure_data.decode().split('\n')[100:110])) # Print a few lines

1 51 LEU
1 52 GLY
1 53 VAL
1 54 PHE
1 55 ALA
1 56 ALA
1 57 LEU
1 58 ALA
1 59 GLU
1 60 ALA


**Getting Confidence Metrics (pLDDT and PAE)** Boltz models provide
confidence metrics compatible with AlphaFold.

- **pLDDT** (predicted Local Distance Difference Test) gives a
  per-residue confidence score from 0-100.
- **PAE** (Predicted Aligned Error) provides an (1x) N x N matrix of expected
  error between every pair of residues.

In [7]:
# Retrieve the pLDDT scores
plddt_scores = fold_job.plddt
print("pLDDT scores shape:", plddt_scores.shape)
print("First 10 scores:", plddt_scores[0, :10])

# Retrieve the PAE matrix
pae_matrix = fold_job.pae
print("\nPAE matrix shape:", pae_matrix.shape)


pLDDT scores shape: (1, 794)
First 10 scores: [0.54034674 0.5691333  0.6142335  0.6112128  0.65294373 0.65395063
 0.6819881  0.77170086 0.8724277  0.95438474]

PAE matrix shape: (1, 794, 794)


**Getting Predicted Binding Affinity** Since we requested it, we can
now retrieve the predicted binding affinity. The result is a
`BoltzAffinity` object containing detailed predictions.

In [8]:
# Retrieve the affinity prediction
affinity_data = fold_job.affinity

print("Affinity for binder 'C':")
print(f"  predicted: {affinity_data.affinity_pred_value}")
print(f"  probability: {affinity_data.affinity_probability_binary}")
print(f"  per model: {affinity_data.per_model}")


Affinity for binder 'C':
  predicted: -1.826643705368042
  probability: 0.99260413646698
  per model: {'affinity_pred_value1': -2.121795654296875, 'affinity_probability_binary1': 0.995862603187561, 'affinity_pred_value2': -1.5314918756484985, 'affinity_probability_binary2': 0.9893457293510437}
